Импортируем модули.

In [111]:
import os
import copy
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression # +
from sklearn.linear_model import Ridge # +
from sklearn.linear_model import Lasso # +
from sklearn.linear_model import LassoLars # +
from sklearn.linear_model import MultiTaskLasso # +
from sklearn.linear_model import ElasticNet # +
from sklearn.linear_model import Perceptron # -
from sklearn.neural_network import MLPRegressor # +
from sklearn.neighbors import KNeighborsRegressor # +
from sklearn.tree import DecisionTreeRegressor # +
from sklearn.svm import SVR # + только для 1 y
from sklearn.svm import LinearSVR # + только для 1 y
from sklearn.ensemble import RandomForestRegressor # +?
from sklearn.ensemble import BaggingRegressor # +?
from sklearn.ensemble import AdaBoostRegressor # +?

Импортируем датасеты (X_bp.xlsx, X_nup.xlsx) и объединяем используя INNER.

In [2]:
x_bp_df = pd.read_excel('../data/X_bp.xlsx', sheet_name='X_bp.csv')
x_nup_df = pd.read_excel('../data/X_nup.xlsx', sheet_name='X_nup.csv')
bp_nup_df = x_bp_df.merge(x_nup_df, left_index=True, right_index=True, how='inner')

Удаляем лишние колонки - индексы датасетов

In [3]:
bp_nup_df.drop(columns = ['Unnamed: 0_x', 'Unnamed: 0_y'],axis = 1, inplace=True)

In [4]:
bp_nup_df.head(5)

,Соотношение матрица-наполнитель,"Плотность, кг/м3","модуль упругости, ГПа","Количество отвердителя, м.%","Содержание эпоксидных групп,%_2","Температура вспышки, С_2","Поверхностная плотность, г/м2","Модуль упругости при растяжении, ГПа","Прочность при растяжении, МПа","Потребление смолы, г/м2","Угол нашивки, град",Шаг нашивки,Плотность нашивки
0,1.857143,2030.0,738.736842,30.00,22.267857,100.000000,210.0,70.0,3000.0,220.0,0.0,4.0,57.0
1,1.857143,2030.0,738.736842,50.00,23.750000,284.615385,210.0,70.0,3000.0,220.0,0.0,4.0,60.0
2,1.857143,2030.0,738.736842,49.90,33.000000,284.615385,210.0,70.0,3000.0,220.0,0.0,4.0,70.0
3,1.857143,2030.0,738.736842,129.00,21.250000,300.000000,210.0,70.0,3000.0,220.0,0.0,5.0,47.0
4,2.771331,2030.0,753.000000,111.86,22.267857,284.615385,210.0,70.0,3000.0,220.0,0.0,5.0,57.0


### Производим классификацию:

Смотрим данные в исходных датасетах.

In [5]:
bp_nup_df['Угол нашивки, град'] = preprocessing.LabelEncoder().fit_transform(bp_nup_df['Угол нашивки, град'])

In [6]:
list(bp_nup_df.columns)

['Соотношение матрица-наполнитель',
 'Плотность, кг/м3',
 'модуль упругости, ГПа',
 'Количество отвердителя, м.%',
 'Содержание эпоксидных групп,%_2',
 'Температура вспышки, С_2',
 'Поверхностная плотность, г/м2',
 'Модуль упругости при растяжении, ГПа',
 'Прочность при растяжении, МПа',
 'Потребление смолы, г/м2',
 'Угол нашивки, град',
 'Шаг нашивки',
 'Плотность нашивки']

In [7]:
df_std_scaler = preprocessing.StandardScaler().fit_transform(bp_nup_df)
df_std_scaler = pd.DataFrame(df_std_scaler, columns = list(bp_nup_df.columns))

In [8]:
df_std_scaler.head(5)

,Соотношение матрица-наполнитель,"Плотность, кг/м3","модуль упругости, ГПа","Количество отвердителя, м.%","Содержание эпоксидных групп,%_2","Температура вспышки, С_2","Поверхностная плотность, г/м2","Модуль упругости при растяжении, ГПа","Прочность при растяжении, МПа","Потребление смолы, г/м2","Угол нашивки, град",Шаг нашивки,Плотность нашивки
0,-1.175779,0.736365,-0.003594,-2.848828,0.009757,-4.542215,-0.969964,-1.06772,1.098244,0.02641,-0.983518,-1.131530,-0.012469
1,-1.175779,0.736365,-0.003594,-2.141666,0.626001,-0.030955,-0.969964,-1.06772,1.098244,0.02641,-0.983518,-1.131530,0.230546
2,-1.175779,0.736365,-0.003594,-2.145202,4.471955,-0.030955,-0.969964,-1.06772,1.098244,0.02641,-0.983518,-1.131530,1.040595
3,-1.175779,0.736365,-0.003594,0.651622,-0.413446,0.344984,-0.969964,-1.06772,1.098244,0.02641,-0.983518,-0.741243,-0.822518
4,-0.174232,0.736365,0.039618,0.045585,0.009757,-0.030955,-0.969964,-1.06772,1.098244,0.02641,-0.983518,-0.741243,-0.012469


In [102]:
data_columns = ['Соотношение матрица-наполнитель', 'Плотность, кг/м3', 'модуль упругости, ГПа', 'Количество отвердителя, м.%',
            'Содержание эпоксидных групп,%_2', 'Температура вспышки, С_2', 'Поверхностная плотность, г/м2',
            'Модуль упругости при растяжении, ГПа', 'Прочность при растяжении, МПа', 'Потребление смолы, г/м2',
            'Угол нашивки, град', 'Шаг нашивки', 'Плотность нашивки']
target = ['Модуль упругости при растяжении, ГПа', 'Прочность при растяжении, МПа']

tdata_columns = ['Соотношение матрица-наполнитель', 'Плотность, кг/м3', 'модуль упругости, ГПа', 'Количество отвердителя, м.%',
            'Содержание эпоксидных групп,%_2', 'Температура вспышки, С_2', 'Поверхностная плотность, г/м2',
            'Прочность при растяжении, МПа', 'Потребление смолы, г/м2',
            'Угол нашивки, град', 'Шаг нашивки', 'Плотность нашивки']
ttarget = ['Модуль упругости при растяжении, ГПа']

ttdata_columns = ['Соотношение матрица-наполнитель', 'Плотность, кг/м3', 'модуль упругости, ГПа', 'Количество отвердителя, м.%',
            'Содержание эпоксидных групп,%_2', 'Температура вспышки, С_2', 'Поверхностная плотность, г/м2',
            'Потребление смолы, г/м2',
            'Угол нашивки, град', 'Шаг нашивки', 'Плотность нашивки']
tttarget = ['Модуль упругости при растяжении, ГПа', 'Прочность при растяжении, МПа']

In [10]:
df_class1 = df_std_scaler[(df_std_scaler['Угол нашивки, град'] >= 0) & (df_std_scaler['Угол нашивки, град'] < 1)]

In [11]:
df_class2 = df_std_scaler[df_std_scaler['Угол нашивки, град'] >= 1]

In [12]:
def issl(df, data, target, tmodel):
    combinations_data = []
    combinations_target = []
    matrix = pd.DataFrame()
    
    if isinstance(df, pd.DataFrame):
        cols_data = tuple(set(data) - set(target))
        cols_target = target
    else:
        raise ValueError("Входные данные должны быть в формате pd.DataFrame")
    
    for _ in range(1,len(cols_data)+1):
        comb_list = list(itertools.combinations(range(len(cols_data)), _))
        combinations_data.append(comb_list)
    
    for _ in range(1,len(cols_target)+1):
        comb_list = list(itertools.combinations(range(len(cols_target)), _))
        combinations_target.append(comb_list)
    
    s = [item for sublist in combinations_target for item in sublist]
    s2 = [item for sublist in combinations_data for item in sublist]
    
    for i in s:
        cols_for_y = [cols_target[i[x]] for x in range(len(i))]
        str_cols_y = '///'.join(cols_for_y)
        matrix[str_cols_y] = ''
        for j in s2:
            cols_for_x = [cols_data[j[x]] for x in range(len(j))]
            str_cols_x = '///'.join(cols_for_x)
            matrix.loc[str_cols_x] = ''
    
    for i in s:
        cols_for_y = [cols_target[i[x]] for x in range(len(i))]
        str_cols_y = '///'.join(cols_for_y)
        y = df[cols_for_y]
        for j in s2:
            cols_for_x = [cols_data[j[x]] for x in range(len(j))]
            str_cols_x = '///'.join(cols_for_x)
            X = df[cols_for_x]
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=5)
            model =  tmodel
            model.fit(X_train, y_train)
            matrix.loc[[str_cols_x], [str_cols_y]] = model.score(X_test, y_test)

    return matrix

In [114]:
# MLPRegressor(max_iter=1100)
XT = df_std_scaler[tdata_columns]
yt = df_std_scaler[ttarget]
XT_train, XT_test, yt_train, yt_test = train_test_split(XT, yt, train_size=0.7, random_state=42)
model_test = MLPRegressor(max_iter=1100)
model_test.fit(XT_train, yt_train) # values.ravel()
model_test.score(XT_test, yt_test)
#yt_test.head(5)

d:\prj\mgtu_ds_u4\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\prj\mgtu_ds_u4\venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1100) reached and the optimization hasn't converged yet.
  warnings.warn(


-1.0407323532337545

In [106]:
# DecisionTreeRegressor()
# KNeighborsRegressor(n_neighbors=3)
# Ridge(alpha=.5)
# LinearRegression()
# LassoLars(alpha=.1, normalize=False)
# MultiTaskLasso(alpha=1.0)
# ElasticNet(alpha=0.1, l1_ratio=0.7)
# 
lr = issl(df_std_scaler, data_columns, target, LinearRegression())
lr.head(1)

,"Модуль упругости при растяжении, ГПа","Прочность при растяжении, МПа","Модуль упругости при растяжении, ГПа///Прочность при растяжении, МПа"
"Угол нашивки, град",-0.003303,-0.02219,-0.012746


In [43]:
lr['Прочность при растяжении, МПа'].min()

-0.022143348681078923

In [48]:
len(lr.index)

2047

In [ ]:
lr.loc['Угол нашивки, град', 'Модуль упругости при растяжении, ГПа']

In [ ]:
lr.iloc[0, [0]]

In [ ]:
yyyy = range(len(lr.index))
plt.scatter(lr['Шаг нашивки'], lr.index)
plt.show()

In [ ]:
y

In [ ]:
dassss = issl(df_minmax_scaler[data_col], df_minmax_scaler[target_col], KNeighborsRegressor(n_neighbors=3))
dassss.head(1)

In [ ]:
dassss['Прочность при растяжении, МПа'].max()

In [ ]:
rd = issl(df_minmax_scaler[data_col], df_minmax_scaler[target_col], Ridge(alpha=.5))
rd.head(1)

In [ ]:
rd['Модуль упругости при растяжении, ГПа///Прочность при растяжении, МПа'].max()

In [ ]:
# reg = Ridge(alpha=.5)
# reg = LinearRegression()
#reg.coef_
X_reg_train = [[0, 0], [1, 1], [2, 2], [2, 4], [5, 5]]
y_reg_train = [0, 1, 2, 3, 10]
X_reg_test = [[0, 0], [1, 1], [2, 2], [2, 4], [5, 5]]
y_reg_test = [0, 1, 2, 3, 10]
reg = Perceptron(fit_intercept=False, max_iter=10, tol=None, shuffle=False)
reg.fit(X_reg_train, y_reg_train)
reg_score = reg.score(X_reg_test, y_reg_test)

In [ ]:
reg_score

In [ ]:
plt.scatter(y_reg_test, y_reg_train)
plt.show()

In [ ]:
lr.to_csv('matrix4.csv')

In [ ]:
matr = pd.DataFrame()
matr['Rows'] = np.nan
#matr.loc['Rowsss'] = ''
#matr.loc[['Rowsss'], ['Test1']] = 12
#matr.loc['Rowsss'] = ''
#matr.loc[['Rowsss'], ['Test12']] = 13
matr.head(5)

In [ ]:
X = df_minmax_scaler[data_col] #[col]
X

In [ ]:
y = df_minmax_scaler[target_col] #'Угол нашивки, град'
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.67, random_state=42)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=16)
knn.fit(X_train, y_train)
print(knn.score(X_test, y_test))

In [ ]:
neigh = KNeighborsRegressor(n_neighbors=2)
neigh.fit(X_train, y_train)
print(neigh.score(X_test, y_test))

In [ ]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
print(dtc.score(X_test, y_test))

In [ ]:
plt.figure(figsize=(20,50))
tree.plot_tree(dtc, feature_names=X.columns);

In [ ]:
dtr = DecisionTreeRegressor()
dtr.fit(X_train, y_train)
print(dtr.score(X_test, y_test))

In [ ]:
plt.figure(figsize=(20,20))
tree.plot_tree(dtr, feature_names=X.columns);

In [ ]:
#KNeighborsRegressor()
bagging = BaggingRegressor(DecisionTreeRegressor(max_depth=4), max_samples=0.67, max_features=0.33).fit(X_train, y_train)

In [ ]:
bagging.score(X_test, y_test)

In [ ]:
rng = np.random.RandomState(1)
clf = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4), n_estimators=100, random_state=rng).fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)